# Geospatial Analysis: Working with MODIS Raster data

- [The Terra and Aqua combined MCD64A1](https://lpdaac.usgs.gov/products/mcd64a1v061/)  Version 6.1 Burned Area data product is a monthly, global gridded 500 meter (m) product containing per-pixel burned-area and quality information

In [3]:
import os
import re
import numpy as np
import numpy.ma as ma
import rasterio as rio
from rasterio.plot import plotting_extent
import earthpy as et
import earthpy.plot as ep
import geopandas
import matplotlib.pyplot as plt
import requests
import dotenv
dotenv.load_dotenv(dotenv.find_dotenv())

True

***list contents of Modis ftp site***

it is no longer a "proper" ftp site so use bs4 to parse.

In [24]:
from bs4 import BeautifulSoup

In [41]:
host = fr'https://e4ftl01.cr.usgs.gov/MOTA/MCD64A1.061'
login = os.getenv('user')
password = os.getenv('pwd')

In [40]:
# list folders 
r = requests.get(host, verify=True, stream=True,auth=(login,password))
soup = BeautifulSoup(r.text, "html.parser")
folders = list()
for link in soup.findAll('a', attrs={'href': re.compile("\d{4}.\d{2}.\d{2}/")}):
    folders.append(link.get('href'))

In [46]:
# list files in folder
for f in folders[:1]:
    file_list = list()
    folder_url = f"{host}/{f}"
    r = requests.get(folder_url, verify=True, stream=True,auth=(login,password))
    soup = BeautifulSoup(r.text, "html.parser")
    for link in soup.findAll('a', attrs={'href': re.compile(".hdf$")}):
        file_list.append(link.get('href'))    

In [57]:
# download file from folder
# implement 25 file timeout to avoid ratelimit
exceptions = list()
for f in folders[:1]:
    for fl in exceptions:
        try:
            file_url = f"{host}/{f}/{fl}"
            r = requests.get(file_url, verify=True, stream=True,auth=(login,password))
            open(f'./data/{fl}',"wb").write(r.content)
        except Exception as error:
            print(error)
            exceptions.append(fl)
            import time
            time.sleep(3)

In [96]:
# USA tiles only
# use modis grid to slice only continents / areas of interest
# https://modis-land.gsfc.nasa.gov/MODLAND_grid.html
hreg = re.compile("h0[8-9]|h1[0-4]")
vreg = re.compile("v0[2-7]")
for fl in file_list:
    h = hreg.search(fl)
    if h:
        print(h.string)
        v = vreg.search(h.string)
        if v:
            print(v.string)

MCD64A1.A2000306.h08v03.061.2021085164213.hdf
MCD64A1.A2000306.h08v03.061.2021085164213.hdf
MCD64A1.A2000306.h08v04.061.2021085165152.hdf
MCD64A1.A2000306.h08v04.061.2021085165152.hdf
MCD64A1.A2000306.h08v05.061.2021085163226.hdf
MCD64A1.A2000306.h08v05.061.2021085163226.hdf
MCD64A1.A2000306.h08v06.061.2021085162824.hdf
MCD64A1.A2000306.h08v06.061.2021085162824.hdf
MCD64A1.A2000306.h08v07.061.2021085171018.hdf
MCD64A1.A2000306.h08v07.061.2021085171018.hdf
MCD64A1.A2000306.h08v08.061.2021085164941.hdf
MCD64A1.A2000306.h08v09.061.2021085163351.hdf
MCD64A1.A2000306.h08v11.061.2021085165322.hdf
MCD64A1.A2000306.h09v02.061.2021085164150.hdf
MCD64A1.A2000306.h09v02.061.2021085164150.hdf
MCD64A1.A2000306.h09v03.061.2021085164808.hdf
MCD64A1.A2000306.h09v03.061.2021085164808.hdf
MCD64A1.A2000306.h09v04.061.2021085171316.hdf
MCD64A1.A2000306.h09v04.061.2021085171316.hdf
MCD64A1.A2000306.h09v05.061.2021085164302.hdf
MCD64A1.A2000306.h09v05.061.2021085164302.hdf
MCD64A1.A2000306.h09v06.061.202108

'MCD64A1.A2000306.h08v03.061.2021085164213.hdf'

***Download a MODIS tile***

In [ ]:
tile = 'MCD19A1.A2020246.h08v05.006.2020270073445'

r = requests.get(url, verify=True, stream=True,auth=(login,password))
open(f'{tile}.hdf',"wb").write(r.content)

***Load Data from MODIS HDF***

<img src="https://www.earthdatascience.org/images/earth-analytics/hierarchical-data-formats/hdf5-example-data-structure-with-metadata.jpg" width="400" height="400">

*HDF files are self describing - this means that all elements (the file itself, groups and datasets) can have associated metadata that describes the information contained within the element. Source: NEON*

In [ ]:
all_bands = []

with rio.open(f'{tile}.hdf') as dataset:
    # capture meta and CRS data
    hdf4_meta = dataset.meta 
    crs = dataset.read_crs()
    
    # iterate data layers and select using name
    for layer_name in [name for name in dataset.subdatasets if 'grid500m:Sur_refl_500m' in name]:
        print(layer_name)
        with rio.open(layer_name) as subdataset:
            modis_meta = subdataset.profile
            all_bands.append(subdataset.read(1))

***Transform Spectral Bands 1-7***

In [ ]:
# Stack pre-fire reflectance bands
pre_fire_modis = np.stack(all_bands)
print(f'Shape: {pre_fire_modis.shape}')
print(f'Meta: {modis_meta}')

In [ ]:
# Mask no data values
pre_fire_modis = ma.masked_where(pre_fire_modis == modis_meta["nodata"], pre_fire_modis)

In [ ]:
# plot
ep.plot_bands(pre_fire_modis,scale=False,cols=3)
plt.show()

In [ ]:
# plot Red, Green and Blue bands
ep.plot_rgb(pre_fire_modis,rgb=[0,3,2]) # RGB bands 1,4,3 (see user guide)
plt.show()

***Write to GeoTiff***

In [ ]:
# prep metadata object
output_meta = modis_meta.copy()
output_meta['driver'] = 'GTiff'
output_meta['count'] = pre_fire_modis.shape[0]
output_meta

In [ ]:
out_path = f'{tile}.tif'
# Export file as a geotiff
with rio.open(out_path, "w", **output_meta) as dest:
    dest.write(pre_fire_modis)

***Load & Reproject GeoTiff***

Geopandas has a convinient global country dataset for plotting.

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
countries = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
NA = countries.loc[countries['continent']=='North America']
NA.plot(ax=ax, edgecolor='green',facecolor='grey',alpha=0.5)
ax.set_xticks(np.arange(-180,0,30))
ax.set_yticks(np.arange(0,90,30))
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.grid(True,color='black',lw=0.5,linestyle='--')
ax.set_title(f"EPSG:{NA.crs.to_epsg()}");

However, in order to plot our MODIS Bands1-7 GeoTiff we will need to reproject the data from EPSG:9122 to the same CRS EPSG:4326 as GeoPandas. We can do this using [rasterio](https://rasterio.readthedocs.io/en/latest/topics/reproject.html) (see also[ this guide from Earth Lab at University of Colorado, Boulder)](https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/raster-data-processing/reproject-raster/).

Altering CRS and reprojection is usefull for many geospatial analysis. The [OSGEO GDAL](https://gdal.org/) "*Geospatial Data Abstraction Library*" is another great tool for reprojection and QGIS provide a nice background on the topic [here](https://docs.qgis.org/3.10/en/docs/training_manual/vector_analysis/reproject_transform.html?highlight=reprojecting)

In [ ]:
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [ ]:
# Set desitnation CRS
# rio.crs.CRS.from_epsg(4326)
dst_crs = f"EPSG:{NA.crs.to_epsg()}"

# set out path
out_path_rproj = out_path.replace('.tif','-4326.tif')

with rio.open(out_path) as src:
    # get src bounds and transform
    transform, width, height = calculate_default_transform(src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({'crs': dst_crs,
                   'transform': transform,
                   'width': width,
                   'height': height})
    
    # reproject and write to file
    with rio.open(out_path_rproj, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(source=rio.band(src, i),
                      destination=rio.band(dst, i),
                      src_transform=src.transform,
                      src_crs=src.crs,
                      dst_transform=transform,
                      dst_crs=dst_crs,
                      resampling=Resampling.nearest)

In [ ]:
# plot the result
fig, axs = plt.subplots(1,2,figsize=(14,7))

raster = rio.open(out_path_rproj)
rio.plot.show((raster,1,4,3), ax=axs[0])
NA.plot(ax=axs[0], edgecolor='green',facecolor='grey',alpha=0.5)
axs[0].set_xticks(np.arange(-180,0,30))
axs[0].set_yticks(np.arange(0,90,30))
axs[0].set_xlabel('Longitude')
axs[0].set_ylabel('Latitude')
axs[0].grid(True,color='black',lw=0.5,linestyle='--')
axs[0].set_title(f"EPSG:{NA.crs.to_epsg()}")

raster = rio.open(out_path_rproj)
rio.plot.show((raster,1,4,3), ax=axs[1])
NA.plot(ax=axs[1], edgecolor='green',facecolor='grey',alpha=0.5)
axs[1].set_xticks(np.arange(-180,0,30))
axs[1].set_yticks(np.arange(0,90,30))
axs[1].set_xlabel('Longitude')
axs[1].set_ylabel('Latitude')
axs[1].grid(True,color='black',lw=0.5,linestyle='--')
axs[1].set_title(f"EPSG:{NA.crs.to_epsg()}")
axs[1].set_xlim(-140,-55)
axs[1].set_ylim(10,50);

***Leaflet***

In [ ]:
from ipyleaflet import Map, basemaps, basemap_to_tiles,ImageOverlay
import matplotlib.pyplot as plt
%matplotlib inline

# !jupyter labextension install jupyter-leaflet
# !jupyter nbextension enable --py --sys-prefix ipyleaflet
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [ ]:
m = Map(basemap=basemap_to_tiles(basemaps.NASAGIBS.ViirsEarthAtNight2012, "2017-04-08"),center=(30,-90),zoom=4)
image = ImageOverlay(url=out_path_rproj,bounds=((raster.bounds[1],raster.bounds[0]),(raster.bounds[3],raster.bounds[2])))
m.add_layer(image)
m

### References
- [Official MODIS authentication Python and R scripts](https://lpdaac.usgs.gov/tools/data-prep-scripts/)